In [0]:
import numpy as np
import time
import os
import random
os.chdir("/Users/anneabeille/Documents/MVA/RL/rubiks_cube")

In [0]:
from cube import *
from network import model

Using TensorFlow backend.


tf.estimator package not installed.


In [0]:
cube1 = cube.Cube()

In [0]:
cube1.issolve()

True

In [0]:
cube1.move("uf")

In [0]:
cube1.labels

array([ 0,  1,  2,  0,  4,  5,  6,  4,  8,  9, 10,  8, 12, 13, 14, 12, 16,
       17, 18, 16, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])

In [0]:
cube1.move('ub')

In [0]:
corners = [0, 1, 2, 3, 8, 9, 10, 11, 16, 17, 18, 19, 28, 29, 30, 31, 36, 37, 38, 39, 40, 41, 42, 43]
edges = [4, 5, 6, 7, 12, 13, 14, 15, 20, 21, 22, 23, 24, 25, 26, 27, 32, 33, 34, 35, 44, 45, 46, 47]

In [0]:
cube1 = cube.Cube()

In [0]:
moves

['uf', 'df', 'rf', 'lf', 'ff', 'bf', 'ub', 'db', 'rb', 'lb', 'fb', 'bb']

In [0]:
cube2, r = cube1.move('ff')

In [0]:
network.model.predict(cube2.reducted_state().reshape(1, 480))

[array([[0.08086485, 0.0818248 , 0.08660321, 0.08502865, 0.07659638,
         0.08321224, 0.09148036, 0.08318015, 0.08550493, 0.08106757,
         0.08019791, 0.08443899]], dtype=float32),
 array([[0.03061193]], dtype=float32)]

In [0]:
#from cube import *
#from network import model

n_shuffle = 3
n_cube = int(64/n_shuffle)
save_path = "/Users/anneabeille/Documents/MVA/RL/rubiks_cube/models/"

def generate_dataset(_model):
    ni = []
    X = np.zeros((n_shuffle*n_cube, 20*24))
    Y_policy = np.zeros((n_shuffle*n_cube, 12))
    Y_value = np.zeros((n_shuffle*n_cube, 1))
    move_list = []
    for nc in range(n_cube):
        _cube = Cube()
        for i in range(n_shuffle):
            _move = moves[random.randint(0, 11)]
            _cube, r = _cube.move(_move)
            children_states = np.zeros((12, 20*24))
            children_rewards = []
            move_list.append(_move)
            for m in range(12):
                child, r_child = _cube.move(moves[m])
                children_states[m, :] = child.reducted_state()
                children_rewards.append(r_child)
            children_rewards = np.array(children_rewards)
            children_states = np.array(children_states)
            children_values = _model.predict(children_states)[1].reshape(12)
            children_values = np.array(children_values)*((children_rewards-1)/(-2)) + np.array(children_rewards)
            print(_move)
            print(children_rewards)
            print(children_values)
            max_pol = np.argmax(children_values)
            max_val = children_values[max_pol]
            max_pol_array = np.zeros(12)
            max_pol_array[max_pol]=1
            ni.append(i+1)
            X[nc*n_shuffle + i, :] = _cube.reducted_state()
            Y_policy[nc*n_shuffle + i, :] = max_pol_array
            Y_value[nc*n_shuffle + i, :] = max_val
    return X, Y_policy, Y_value, ni, move_list

In [0]:
_cube = Cube()
_move = moves[random.randint(0, 11)]
_cube, r = _cube.move(_move)

children_states = np.zeros((12, 20*24))
children_rewards = []
for m in range(12):
    child, r_child = _cube.move(moves[m])
    children_states[m, :] = child.reducted_state()
    children_rewards.append(r_child)
children_rewards = np.array(children_rewards)
children_states = np.array(children_states)
children_values = model.predict(children_states)[1].reshape(12)
children_values = np.array(children_values)*((children_rewards-1)/(-2)) + np.array(children_rewards)
max_pol = np.argmax(children_values)
max_val = children_values[max_pol]
max_pol_array = np.zeros(12)
max_pol_array[max_pol]=1

X = _cube.reducted_state()
Y_policy = max_pol_array
Y_value = max_val

In [0]:
X, Y_policy, Y_value, ni, move_list = generate_dataset(model)

In [0]:
cube1 = Cube()
cube1.labels

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])

In [0]:
moves = ["uf", "df", "rf", "lf", "ff", "bf", "ub", "db", "rb", "lb", "fb", "bb"]

In [0]:
cube1 = Cube()
cube2, r2 = cube1.move("uf")
cube3, r3 = cube2.move("ub")
print(r2)
print(r3)

-1
1


In [0]:
cube3.issolve()

True

In [0]:
cube3.labels

array([ 0, 42, 41,  3,  4, 13,  6, 35,  8, 37, 36, 11, 12,  5, 14, 47, 30,
       29, 18, 19, 25, 24, 23, 20, 21, 26, 27, 22, 28, 40, 16, 31, 32,  7,
       34, 45, 10,  9, 39, 38, 17,  2,  1, 43, 44, 15, 46, 33])

In [0]:
cube1 = Cube()
cube2, r = cube1.move('lf')
for m in range(12):
    child, r_child = cube2.move(moves[m])
    print(r_child)

-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1


In [0]:
model.predict(cube2.reducted_state().reshape((1, 480)))

[array([[6.8640446e-05, 1.1812518e-04, 1.2448260e-05, 2.5434998e-05,
         2.8276557e-04, 6.4926485e-06, 4.5832887e-05, 2.5799422e-05,
         4.8715905e-05, 5.2217605e-05, 9.9930537e-01, 8.0626269e-06]],
       dtype=float32), array([[1.0044404]], dtype=float32)]

In [0]:
cube1 = Cube()
cube2, r = cube1.move('rf')
print(r)
cube3, r = cube2.move('uf')
print(r)
cube4, r = cube3.move('uf')
print(r)
cube5, r = cube4.move('rb')
print(r)
print(cube1.labels)
print(cube2.labels)
print(cube3.labels)
print(cube4.labels)
print(cube5.labels)

-1
-1
-1
-1
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
[ 0 36 16  3  4 24  6  7  8 29  1 11 12 21 14 15 41  9 18 19 20 33  5 23
 45 13 26 27 28 37  2 31 32 25 34 35 42 17 38 39 40 30 10 43 44 22 46 47]
[36 16  3  0 24  6  7  4 29  1 11  8 21 14 15 12  9 18 19 41 20 33  5 23
 45 13 26 27 28 37  2 31 32 25 34 35 42 17 38 39 40 30 10 43 44 22 46 47]
[16  3  0 36  6  7  4 24  1 11  8 29 14 15 12 21 18 19 41  9 20 33  5 23
 45 13 26 27 28 37  2 31 32 25 34 35 42 17 38 39 40 30 10 43 44 22 46 47]
[16  8  2 36  6  5  4 24  1 19 10 29 14 13 12 21  0 17 41  9 20 15 22 23
  7 25 26 27 28 11 30 31 32 33 34 35  3 37 38 39 40 18 42 43 44 45 46 47]


In [0]:
model.predict(cube4.reducted_state().reshape((1, 480)))

[array([[6.6360664e-01, 2.7150961e-03, 9.2630682e-05, 2.7735604e-04,
         2.0531680e-04, 1.4531509e-04, 3.2520136e-01, 4.9282708e-03,
         2.0160030e-03, 4.8887666e-04, 1.8547697e-04, 1.3764430e-04]],
       dtype=float32), array([[-1.0077515]], dtype=float32)]

In [0]:
X, Y_policy, Y_value, ni, move_list = generate_dataset(model)

ff
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1]
[-3.28322053e-02 -8.81671906e-03 -2.43158340e-02 -4.37724590e-03
 -5.08981347e-02  3.33559513e-03 -2.08745599e-02 -2.85819173e-02
  6.77704811e-04 -4.21938896e-02  1.00000000e+00  4.82273102e-03]
lb
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-0.00820231 -0.01287127  0.00357783  0.21304584 -0.00117618 -0.03762901
  0.00766838 -0.05977833  0.00348508 -0.09753501  0.02343953 -0.05578339]
lf
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-0.01203579 -0.01155996  0.00357783 -0.00437725 -0.03730381  0.0033356
 -0.00042677  0.00172019  0.00348508 -0.04219389  0.10171831  0.00482273]
lf
[-1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1]
[-0.01859522 -0.0209105   0.00357783 -0.12523735 -0.04516369 -0.0638333
 -0.05252057 -0.01143682  0.00348508  1.         -0.01495981 -0.05446601]
bb
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[ 0.00048327 -0.04155338 -0.03637254 -0.03018129  0.00444043  0.1845578
 -0.02889198 -0.01226604 -0.0481497  -0.07724267  0.0035795  -0.04250497]
rb
[-1 -1 -1 -

uf
[-1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1]
[-0.04997444  0.00247562 -0.03006375 -0.01189613 -0.03831822 -0.01810789
  1.          0.00404561 -0.03601384 -0.02498394 -0.01530874 -0.08796299]
lb
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[ 0.02189863 -0.03155041  0.00357783  0.20764077 -0.00846195  0.01367939
 -0.04151106 -0.05042499  0.00348508 -0.1018188   0.01565158 -0.09097922]
lb
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-0.0348376  -0.01190734  0.00357783 -0.02498394  0.01272309 -0.0227567
 -0.0113095  -0.01678276  0.00348508 -0.01189613  0.005759   -0.07117009]
ub
[ 1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[ 1.          0.00247562 -0.06961238  0.01866353 -0.06140614 -0.03061402
 -0.04997444  0.00404561 -0.02890742  0.00680292 -0.00796723 -0.05204546]
ff
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-0.01506215 -0.00093448  0.01224613 -0.00437725 -0.04750073  0.0033356
 -0.02355123 -0.05468917 -0.04790199 -0.04219389  0.19086349  0.00482273]
lf
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-0.01859522 -0.05362

ub
[ 1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[ 1.          0.00247562 -0.06961238  0.01866353 -0.06140614 -0.03061402
 -0.04997444  0.00404561 -0.02890742  0.00680292 -0.00796723 -0.05204546]
rf
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-0.02267551 -0.01783478 -0.07088691  0.00436556  0.0389291  -0.03725231
 -0.02740026 -0.05484962  0.16923583  0.0048753  -0.0218941   0.03394032]
df
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[ 0.00471354 -0.08523703 -0.00028628 -0.03692406  0.04045606 -0.05491984
  0.00380254  0.161376   -0.06478918 -0.0358392   0.01205373 -0.01540279]
bb
[-1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1]
[-0.02232969 -0.04155338 -0.05984056 -0.03405654  0.00444043  1.
 -0.03554368 -0.01226604 -0.04431129 -0.08007801  0.0035795  -0.10701907]
db
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[ 0.00471354  0.10912037 -0.01768708 -0.00502276  0.02736998 -0.02585232
  0.00380254 -0.10638714 -0.05444324 -0.00077963  0.03242242 -0.06455308]
uf
[-1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1]
[-0.04997444  0.00247562 -0


[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1]
[-0.01988471 -0.09412265 -0.0531348  -0.04219079  0.00444043 -0.10701907
 -0.0612818  -0.05072165 -0.07449174 -0.05782896  0.0035795   1.        ]
ub
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[ 0.18323004  0.00247562 -0.05113041  0.01866353 -0.04697949  0.01205826
 -0.01712143  0.00404561 -0.0263651   0.00680292 -0.00654256 -0.0095734 ]
df
[-1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1]
[ 0.00471354 -0.11880416  0.00887764 -0.00962168  0.04045606 -0.07363957
  0.00380254  1.         -0.06647676 -0.02332377  0.01205373 -0.01929635]
ub
[ 1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[ 1.          0.00247562 -0.06961238  0.01866353 -0.06140614 -0.03061402
 -0.04997444  0.00404561 -0.02890742  0.00680292 -0.00796723 -0.05204546]
fb
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-0.02947235 -0.03597605  0.01530194 -0.07628012  0.19086349  0.0033356
 -0.02300775 -0.03721356 -0.08812714  0.05423498 -0.04750073  0.00482273]
bf
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1]
[-0.01988471 -0.094122

fb
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-0.04712546 -0.02842677  0.01530194 -0.04810411  0.18991244  0.0033356
 -0.05199182 -0.03159326 -0.08812714 -0.02601945 -0.03888798  0.00482273]
bf
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1]
[-0.01988471 -0.09412265 -0.0531348  -0.04219079  0.00444043 -0.10701907
 -0.0612818  -0.05072165 -0.07449174 -0.05782896  0.0035795   1.        ]
bf
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1]
[-0.01988471 -0.09412265 -0.0531348  -0.04219079  0.00444043 -0.10701907
 -0.0612818  -0.05072165 -0.07449174 -0.05782896  0.0035795   1.        ]
bf
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-0.05876362 -0.04835141 -0.02254736 -0.00361753  0.00444043  0.00482273
 -0.03337181 -0.00826263 -0.05188072 -0.04267275  0.0035795   0.0033356 ]
lf
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[ 0.0097146  -0.0209105   0.00357783 -0.07691664 -0.05424964 -0.04236054
 -0.02656853 -0.01143682  0.00348508  0.14611661  0.00758815 -0.05850828]
lb
[-1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1]
[ 0.01424408 -0.0128

In [0]:
for i in range(n_shuffle):
    print(Y_policy[i], Y_value[i], ni[i], move_list[i])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] [1.] 1 ff
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] [0.21304584] 2 lb
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] [0.10171831] 3 lf


In [0]:
print(Y_policy[1], Y_value[2], ni[0], move_list[0])

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])